In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
data_express = pd.read_excel('./Datas/Express.xlsx')
data_shipments = pd.read_csv('./Datas/Shipments2.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Tratando Dados Shipments

In [3]:
data_shipments.head()

,to_postcode,to_country_id
0,00000000,32
1,00005-820,32
2,01031-970,32
3,01033001,64
4,01033-001,32


In [4]:
data_shipments = data_shipments.rename(columns = {'to_postcode' : 'CEP'})

In [5]:
data_shipments.head()

,CEP,to_country_id
0,00000000,32
1,00005-820,32
2,01031-970,32
3,01033001,64
4,01033-001,32


In [6]:
data_shipments['CEP'] = data_shipments['CEP'].replace(to_replace=r'\D+', value = '', regex = True)

In [7]:
data_shipments.head()

,CEP,to_country_id
0,00000000,32
1,00005820,32
2,01031970,32
3,01033001,64
4,01033001,32


In [8]:
data_shipments = data_shipments.loc[data_shipments['to_country_id'] == 32]

In [9]:
data_shipments.head()

,CEP,to_country_id
0,00000000,32
1,00005820,32
2,01031970,32
4,01033001,32
7,01041000,32


In [10]:
data_shipments = data_shipments.reset_index(drop = True)

In [11]:
data_shipments.head()

,CEP,to_country_id
0,00000000,32
1,00005820,32
2,01031970,32
3,01033001,32
4,01041000,32


In [12]:
data_shipments['CEP'] = data_shipments['CEP'].str.strip()

In [13]:
data_shipments['CEP'] = data_shipments['CEP'].replace('', np.nan)

In [14]:
valor_padrao = 0
data_shipments = data_shipments.fillna(valor_padrao)

In [15]:
data_shipments.loc[data_shipments['CEP'] == 0]

,CEP,to_country_id
11505,0,32


In [16]:
data_shipments.dtypes

CEP              object
to_country_id     int64
dtype: object

# Tratando dados Express

In [17]:
data_express.head()

,CEP INI,CEP FIN,NOME COBERTURA,UF,FRANQUIA,EXPRESSO
0,16800000,16849999,MIRANDOPOLIS,SP,AAN,ATENDIDO
1,16900001,16905265,ANDRADINA,SP,AAN,ATENDIDO
2,16920000,16939999,CASTILHO,SP,AAN,ATENDIDO
3,16950000,16979999,MURUTINGA DO SUL,SP,AAN,ATENDIDO
4,16980000,16999999,GUARACAI,SP,AAN,ATENDIDO


In [19]:
data_express = data_express.rename(columns = {'CEP INI' : 'CEP_INI'})
data_express = data_express.rename(columns = {'CEP FIN' : 'CEP_FIN'})

In [20]:
data_express.head()

,CEP_INI,CEP_FIN,NOME COBERTURA,UF,FRANQUIA,EXPRESSO
0,16800000,16849999,MIRANDOPOLIS,SP,AAN,ATENDIDO
1,16900001,16905265,ANDRADINA,SP,AAN,ATENDIDO
2,16920000,16939999,CASTILHO,SP,AAN,ATENDIDO
3,16950000,16979999,MURUTINGA DO SUL,SP,AAN,ATENDIDO
4,16980000,16999999,GUARACAI,SP,AAN,ATENDIDO


In [21]:
data_express['CEP_RANGE'] = [list(range(CEP_INI, CEP_FIN + 1)) for CEP_INI, CEP_FIN in zip(data_express['CEP_INI'], data_express['CEP_FIN'])]

In [22]:
data_express.head()

,CEP_INI,CEP_FIN,NOME COBERTURA,UF,FRANQUIA,EXPRESSO,CEP_RANGE
0,16800000,16849999,MIRANDOPOLIS,SP,AAN,ATENDIDO,"[16800000, 16800001, 16800002, 16800003, 16800..."
1,16900001,16905265,ANDRADINA,SP,AAN,ATENDIDO,"[16900001, 16900002, 16900003, 16900004, 16900..."
2,16920000,16939999,CASTILHO,SP,AAN,ATENDIDO,"[16920000, 16920001, 16920002, 16920003, 16920..."
3,16950000,16979999,MURUTINGA DO SUL,SP,AAN,ATENDIDO,"[16950000, 16950001, 16950002, 16950003, 16950..."
4,16980000,16999999,GUARACAI,SP,AAN,ATENDIDO,"[16980000, 16980001, 16980002, 16980003, 16980..."


In [23]:
# Criando as Lista para CEP (Shipments) e CEP_RANGE (Express)

In [24]:
express_cep_range = data_express['CEP_RANGE'].explode().tolist()

In [28]:
express_cep_range[1]

16800001

In [32]:
express_cep_range = list(map(str, express_cep_range))

In [33]:
express_cep_range[1]

'16800001'

In [34]:
shipments_cep = data_shipments['CEP'].explode().tolist()

In [35]:
shipments_cep[1]

'00005820'

### Verificando os CEPs Shipments com CEPs Express

In [36]:
#Otimziando busca de dados
express_cep_range_set = set(express_cep_range)
shipments_cep_set = set(shipments_cep)

In [40]:
len(express_cep_range)

51801245

In [41]:
len(express_cep_range_set)

51801245

In [42]:
len(shipments_cep)

11506

In [43]:
len(shipments_cep_set)

11070

In [44]:
valor_comparar = 0
for valor in shipments_cep_set:
    if valor in express_cep_range_set:
        valor_comparar += 1

In [47]:
porcentagem = round((valor_comparar / len(shipments_cep_set)) * 100,2)

In [48]:
porcentagem

68.55

In [49]:
print(f'A porcentagem estimada de CEPs Shipments dentro de CEPs Express é aprox. {porcentagem}%')

A porcentagem estimada de CEPs Shipments dentro de CEPs Express é aprox. 68.55%
